In [1]:
import numpy as np
import os
from  natsort import natsorted
import imageio
import re
import time
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input, GRU, Reshape
from keras.layers import Conv2D, MaxPooling2D, Bidirectional, Permute, LSTM, SeparableConv2D
from keras.callbacks import ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers.merge import add, concatenate
from keras import backend as K
from keras.models import load_model, Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import itertools
import editdistance
from collections import Counter

Using TensorFlow backend.


In [2]:
NAME = 'ALPR_CNN'
data_dir = 'Data'
data_dir_2 = 'Data/dataset_1'
model_dir = 'Models'

In [3]:
def prepare_dataset(data_dir, folder_name):
    try:
        print('Loading numpy')
        X = np.load(os.path.join(data_dir,'X_{}.npy'.format(folder_name)))
        y = np.load(os.path.join(data_dir,'y_{}.npy'.format(folder_name)))

    except:
        print('Loading images')
        image_list = []
        labels = []
        pictures_dir = os.path.join(data_dir, folder_name)
        names = [ d for d in os.listdir( pictures_dir ) if d.endswith( '.jpg') ]
        names = natsorted(names)
        for image in names:
            image_list.append(imageio.imread(os.path.join(pictures_dir, image)))
            label = re.split('[._]', image)
            labels.append(label[0][2:])
            print(label[0][2:])
        X = np.stack(image_list, axis=0)
        y = np.array(labels)
        np.save(os.path.join(data_dir,'X_{}'.format(folder_name)),X)
        np.save(os.path.join(data_dir,'y_{}'.format(folder_name)),y)
    return X,y


def prepare_dataset_2(data_dir, folder_name):
    try:
        print('Loading numpy')
        X = np.load(os.path.join(data_dir,'X_{}.npy'.format(folder_name)))
        y = np.load(os.path.join(data_dir,'y_{}.npy'.format(folder_name)))

    except:
        print('Loading images')
        image_list = []
        labels = []
        pictures_dir = os.path.join(data_dir, folder_name)
        names = [ d for d in os.listdir( pictures_dir ) if d.endswith( '.jpg') ]
        names = natsorted(names)
        for image in names:
            image_list.append(imageio.imread(os.path.join(pictures_dir, image)))
            label = re.split('[._]', image)
            labels.append(label[1])
            print(label[1])
        X = np.stack(image_list, axis=0)
        y = np.array(labels)
        np.save(os.path.join(data_dir,'X_{}'.format(folder_name)),X)
        np.save(os.path.join(data_dir,'y_{}'.format(folder_name)),y)
    return X,y

In [4]:
X1,y1 = prepare_dataset(data_dir, 'resized')
print(X1.shape)
print(y1.shape)

Loading numpy
(3921, 100, 160, 3)
(3921,)


In [5]:
X2,y2 = prepare_dataset_2(data_dir_2, 'Resized')
print(X2.shape)
print(y2.shape)

Loading numpy
(300, 100, 160, 3)
(300,)


In [6]:
X_train = np.concatenate((X1,X2), axis=0)
y_train = np.concatenate((y1,y2), axis=0)
print(X_train.shape)
print(y_train.shape)

(4221, 100, 160, 3)
(4221,)


In [7]:
X,y = prepare_dataset_2(data_dir_2, 'Resized_test')
print(X.shape)
print(y.shape)

Loading numpy
(1749, 100, 160, 3)
(1749,)


In [8]:
np.random.seed(42)
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)

X_train = X_train[indices]
y_train = y_train[indices]

In [9]:
np.random.seed(42)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

In [10]:
z = 1200
X_test = X[:z]
X_val = X[z:]
y_test = y[:z]
y_val = y[z:]

In [11]:
regex = r'^[A-Z0-9 ]+$'
alphabet = u'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '

In [12]:
# Translation of characters to unique integer values
def text_to_labels(text):
    ret = []
    for char in text:
        ret.append(alphabet.find(char))
    return ret


# Reverse translation of numerical classes back to characters
def labels_to_text(labels):
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)


# only a-z and space..probably not to difficult
# to expand to uppercase and symbols

def is_valid_str(in_str):
    search = re.compile(regex, re.UNICODE).search
    return bool(search(in_str))

In [13]:
#Removing 7 label plates
mylen = np.vectorize(len)

count = mylen(y_train)
X_train = np.delete(X_train, np.where(count!=6)[0], axis=0 )
y_train = np.delete(y_train, np.where(count!=6)[0], axis=0 )

count = mylen(y_val)
X_val = np.delete(X_val, np.where(count!=6)[0], axis=0 )
y_val = np.delete(y_val, np.where(count!=6)[0], axis=0 )

count = mylen(y_test)
X_test = np.delete(X_test, np.where(count!=6)[0], axis=0 )
y_test = np.delete(y_test, np.where(count!=6)[0], axis=0 )

In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(4215, 100, 160, 3)
(4215,)
(543, 100, 160, 3)
(543,)
(1197, 100, 160, 3)
(1197,)


In [15]:
def code_labels(labels):
    y_coded = []
    for l in labels:
        y_coded.append(text_to_labels(l))
    return np.array(y_coded)

In [16]:
y_train_coded = code_labels(y_train)
y_val_coded = code_labels(y_val)
y_test_coded = code_labels(y_test)
print(y_train_coded.shape)
print(y_val_coded.shape)
print(y_test_coded.shape)

(4215, 6)
(543, 6)
(1197, 6)


In [17]:
num_classes = 36
#Labels to binary
y_train_0 = keras.utils.to_categorical(y_train_coded[:,0],num_classes)
y_train_1 = keras.utils.to_categorical(y_train_coded[:,1],num_classes)
y_train_2 = keras.utils.to_categorical(y_train_coded[:,2],num_classes)
y_train_3 = keras.utils.to_categorical(y_train_coded[:,3],num_classes)
y_train_4 = keras.utils.to_categorical(y_train_coded[:,4],num_classes)
y_train_5 = keras.utils.to_categorical(y_train_coded[:,5],num_classes)

y_val_0 = keras.utils.to_categorical(y_val_coded[:,0],num_classes)
y_val_1 = keras.utils.to_categorical(y_val_coded[:,1],num_classes)
y_val_2 = keras.utils.to_categorical(y_val_coded[:,2],num_classes)
y_val_3 = keras.utils.to_categorical(y_val_coded[:,3],num_classes)
y_val_4 = keras.utils.to_categorical(y_val_coded[:,4],num_classes)
y_val_5 = keras.utils.to_categorical(y_val_coded[:,5],num_classes)

y_test_0 = keras.utils.to_categorical(y_test_coded[:,0],num_classes)
y_test_1 = keras.utils.to_categorical(y_test_coded[:,1],num_classes)
y_test_2 = keras.utils.to_categorical(y_test_coded[:,2],num_classes)
y_test_3 = keras.utils.to_categorical(y_test_coded[:,3],num_classes)
y_test_4 = keras.utils.to_categorical(y_test_coded[:,4],num_classes)
y_test_5 = keras.utils.to_categorical(y_test_coded[:,5],num_classes)


## Keras Model

In [18]:
img_w = 100
# Input Parameters
img_h = 160

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_w, img_h)
else:
    input_shape = (img_w, img_h, 3)


In [19]:
def CNN_model_1(input_shape, p=0.3):
    input_data = Input(name='the_input', shape=input_shape, dtype='float32')
    x = Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=input_shape,
                   padding='same', name='Conv_1')(input_data)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_1')(x)
    x = BatchNormalization(name='Bn_1')(x)
    x = Dropout(p, name='Drop_1')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_2')(x)
    x = BatchNormalization(name='Bn_2')(x)
    x = Dropout(p, name='Drop_2')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_3')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_3')(x)
    x = BatchNormalization(name='Bn_3')(x)
    x = Dropout(p, name='Drop_3')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_4')(x)
    x = BatchNormalization(name='Bn_4')(x)
    x = Dropout(p, name='Drop_4')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_5')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_5')(x)
    x = BatchNormalization(name='Bn_5')(x)
    x = Dropout(p, name='Drop_5')(x)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_6')(x)
    x = BatchNormalization(name='Bn_6')(x)
    x = Dropout(p, name='Drop_6')(x)
    x = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_7')(x)
    x = BatchNormalization(name='Bn_7')(x)
    x = Dropout(p, name='Drop_7')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='dense_1')(x)
    x = BatchNormalization(name='Bn_8')(x)
    x = Dropout(p, name='Drop_8')(x)

    #output
    d1 = Dense(num_classes, activation='softmax', name='dense_out_1')(x)
    d2 = Dense(num_classes, activation='softmax', name='dense_out_2' )(x)
    d3 = Dense(num_classes, activation='softmax', name='dense_out_3')(x)
    d4 = Dense(num_classes, activation='softmax', name='dense_out_4')(x)
    d5 = Dense(num_classes, activation='softmax', name='dense_out_5')(x)
    d6 = Dense(num_classes, activation='softmax', name='dense_out_6')(x)
    model = Model(input_data, [d1,d2,d3,d4,d5,d6])
    return model


def CNN_model_2(input_shape, p=0.3):
    input_data = Input(name='the_input', shape=input_shape, dtype='float32')
    x = Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=input_shape,
                   padding='same', name='Conv_1')(input_data)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_1')(x)
    x = BatchNormalization(name='Bn_1')(x)
    x = Dropout(p, name='Drop_1')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_2')(x)
    x = BatchNormalization(name='Bn_2')(x)
    x = Dropout(p, name='Drop_2')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_3')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_3')(x)
    x = BatchNormalization(name='Bn_3')(x)
    x = Dropout(p, name='Drop_3')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_4')(x)
    x = BatchNormalization(name='Bn_4')(x)
    x = Dropout(p, name='Drop_4')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_5')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_5')(x)
    x = BatchNormalization(name='Bn_5')(x)
    x = Dropout(p, name='Drop_5')(x)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_6')(x)
    x = BatchNormalization(name='Bn_6')(x)
    x = Dropout(p, name='Drop_6')(x)
    x = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_7')(x)
    x = BatchNormalization(name='Bn_7')(x)
    x = Dropout(p, name='Drop_7')(x)
    x = Flatten()(x)
    

    #output
    d1= Dense(128, activation='relu', name='dense_d1')(x)
    d1 = BatchNormalization(name='Bn_d1')(d1)
    d1 = Dropout(p, name='Drop_d1')(d1)
    d1 = Dense(num_classes, activation='softmax', name='dense_out_1')(d1)
    d2 = Dense(128, activation='relu', name='dense_d2')(x)
    d2 = BatchNormalization(name='Bn_d2')(d2)
    d2 = Dropout(p, name='Drop_d2')(d2)
    d2 = Dense(num_classes, activation='softmax', name='dense_out_2' )(d2)
    d3 = Dense(128, activation='relu', name='dense_d3')(x)
    d3 = BatchNormalization(name='Bn_d3')(d3)
    d3 = Dropout(p, name='Drop_d3')(d3)
    d3 = Dense(num_classes, activation='softmax', name='dense_out_3')(d3)
    d4 = Dense(128, activation='relu', name='dense_d4')(x)
    d4 = BatchNormalization(name='Bn_d4')(d4)
    d4 = Dropout(p, name='Drop_d4')(d4)
    d4 = Dense(num_classes, activation='softmax', name='dense_out_4')(d4)
    d5 = Dense(128, activation='relu', name='dense_d5')(x)
    d5 = BatchNormalization(name='Bn_d5')(d5)
    d5 = Dropout(p, name='Drop_d5')(d5)
    d5 = Dense(num_classes, activation='softmax', name='dense_out_5')(d5)
    d6 = Dense(128, activation='relu', name='dense_d6')(x)
    d6 = BatchNormalization(name='Bn_d6')(d6)
    d6 = Dropout(p, name='Drop_d6')(d6)
    d6 = Dense(num_classes, activation='softmax', name='dense_out_6')(d6)
    model = Model(input_data, [d1,d2,d3,d4,d5,d6])
    return model

def ConvLSTM_model_1(input_shape, p=0.3):
    input_data = Input(name='the_input', shape=input_shape, dtype='float32')
    x = Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=input_shape,
                   padding='same', name='Conv_1')(input_data)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_1')(x)
    x = BatchNormalization(name='Bn_1')(x)
    x = Dropout(p, name='Drop_1')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_2')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_2')(x)
    x = BatchNormalization(name='Bn_2')(x)
    x = Dropout(p, name='Drop_2')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_3')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_3')(x)
    x = BatchNormalization(name='Bn_3')(x)
    x = Dropout(p, name='Drop_3')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_4')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_4')(x)
    x = BatchNormalization(name='Bn_4')(x)
    x = Dropout(p, name='Drop_4')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_5')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_5')(x)
    x = BatchNormalization(name='Bn_5')(x)
    x = Dropout(p, name='Drop_5')(x)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_6')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_6')(x)
    x = BatchNormalization(name='Bn_6')(x)
    x = Dropout(p, name='Drop_6')(x)
    x = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_7')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_7')(x)
    x = BatchNormalization(name='Bn_7')(x)
    x = Dropout(p, name='Drop_7')(x)
    x = Permute((2, 1, 3), name='Permute_1')(x)  # for swap-dimension
    x = Reshape((-1, 16 * (img_w // (2 ** 2))), name='Reshape_1')(x)
    x = Dense(32, activation='relu', name='dense1')(x)
    x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
    x = BatchNormalization(name='Bn_LSTM1')(x)
    x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
    x = BatchNormalization(name='Bn_LSTM2')(x)
    x = Dropout(p, name='Drop_14')(x)
    x = Dense(128, activation='relu', name='dense_2')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='dense_1')(x)
    x = BatchNormalization(name='Bn_8')(x)
    x = Dropout(p, name='Drop_8')(x)

    #output
    d1 = Dense(num_classes, activation='softmax', name='dense_out_1')(x)
    d2 = Dense(num_classes, activation='softmax', name='dense_out_2' )(x)
    d3 = Dense(num_classes, activation='softmax', name='dense_out_3')(x)
    d4 = Dense(num_classes, activation='softmax', name='dense_out_4')(x)
    d5 = Dense(num_classes, activation='softmax', name='dense_out_5')(x)
    d6 = Dense(num_classes, activation='softmax', name='dense_out_6')(x)
    model = Model(input_data, [d1,d2,d3,d4,d5,d6])
    return model

def ConvLSTM_model_ds(input_shape, p=0.3):
    input_data = Input(name='the_input', shape=input_shape, dtype='float32')
    x = SeparableConv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=input_shape,
                   padding='same', name='Conv_1')(input_data)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_1')(x)
    x = BatchNormalization(name='Bn_1')(x)
    x = Dropout(p, name='Drop_1')(x)
    x = SeparableConv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_2')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_2')(x)
    x = BatchNormalization(name='Bn_2')(x)
    x = Dropout(p, name='Drop_2')(x)
    x = SeparableConv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_3')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_3')(x)
    x = BatchNormalization(name='Bn_3')(x)
    x = Dropout(p, name='Drop_3')(x)
    x = SeparableConv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_4')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_4')(x)
    x = BatchNormalization(name='Bn_4')(x)
    x = Dropout(p, name='Drop_4')(x)
    x = SeparableConv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_5')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_5')(x)
    x = BatchNormalization(name='Bn_5')(x)
    x = Dropout(p, name='Drop_5')(x)
    x = SeparableConv2D(32, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_6')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_6')(x)
    x = BatchNormalization(name='Bn_6')(x)
    x = Dropout(p, name='Drop_6')(x)
    x = SeparableConv2D(16, kernel_size=(3, 3), activation='relu', padding='same', name='Conv_7')(x)
    #x = MaxPooling2D(pool_size=(2, 2), name='Max_pool_7')(x)
    x = BatchNormalization(name='Bn_7')(x)
    x = Dropout(p, name='Drop_7')(x)
    x = Permute((2, 1, 3), name='Permute_1')(x)  # for swap-dimension
    x = Reshape((-1, 16 * (img_w // (2 ** 2))), name='Reshape_1')(x)
    x = Dense(32, activation='relu', name='dense1')(x)
    x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
    x = BatchNormalization(name='Bn_LSTM1')(x)
    x = Bidirectional(LSTM(32, return_sequences=True, stateful=False, name='Lstm_1'))(x)
    x = BatchNormalization(name='Bn_LSTM2')(x)
    x = Dropout(p, name='Drop_14')(x)
    x = Dense(128, activation='relu', name='dense_2')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='dense_1')(x)
    x = BatchNormalization(name='Bn_8')(x)
    x = Dropout(p, name='Drop_8')(x)

    #output
    d1 = Dense(num_classes, activation='softmax', name='dense_out_1')(x)
    d2 = Dense(num_classes, activation='softmax', name='dense_out_2' )(x)
    d3 = Dense(num_classes, activation='softmax', name='dense_out_3')(x)
    d4 = Dense(num_classes, activation='softmax', name='dense_out_4')(x)
    d5 = Dense(num_classes, activation='softmax', name='dense_out_5')(x)
    d6 = Dense(num_classes, activation='softmax', name='dense_out_6')(x)
    model = Model(input_data, [d1,d2,d3,d4,d5,d6])
    return model

In [20]:
model = ConvLSTM_model_ds(input_shape, p=0.3)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 100, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv_1 (SeparableConv2D)        (None, 100, 160, 32) 155         the_input[0][0]                  
__________________________________________________________________________________________________
Max_pool_1 (MaxPooling2D)       (None, 50, 80, 32)   0           Conv_1[0][0]                     
__________________________________________________________________________________________________
Bn_1 (BatchNormalization)       (None, 50, 80, 32)   128         Max_pool_1[0][0]                 
__________________________________________________________________________________________________
Drop_1 (Dr

In [21]:
def saveModel(epoch, logs):
    global best_val_acc
    global chk_path
    val_dense_out_1_acc = logs['val_dense_out_1_acc']
    val_dense_out_2_acc = logs['val_dense_out_2_acc']
    val_dense_out_3_acc = logs['val_dense_out_3_acc']
    val_dense_out_4_acc = logs['val_dense_out_4_acc']
    val_dense_out_5_acc = logs['val_dense_out_5_acc']
    val_dense_out_6_acc = logs['val_dense_out_6_acc']
    val_acc = np.mean([val_dense_out_1_acc, val_dense_out_2_acc, val_dense_out_3_acc,
                      val_dense_out_4_acc, val_dense_out_5_acc, val_dense_out_6_acc])

    if val_acc > best_val_acc:
        print('Avg_val_acc imporved from {} to {}'.format(best_val_acc, val_acc))
        print('Saved model {}'.format(chk_path))
        best_val_acc = val_acc
        model.save(chk_path)

#callbacks = [LambdaCallback(on_epoch_end=saveModel)]

In [26]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

batch_size = 50
epochs = 500
NAME = 'ConvLSTM_model_1'

model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
t = int(time.time())

# checkpoint
best_val_acc = 0
chk_path = os.path.join(model_dir, 'best_{}_{}'.format(NAME,t))
# checkpoint = ModelCheckpoint(chk_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpoint = LambdaCallback(on_epoch_end=saveModel)
tensorboard = TensorBoard(log_dir="logs/{}_{}".format(NAME,t))
callbacks_list = [checkpoint, tensorboard]

history = model.fit(X_train, [y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5],
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            shuffle=True,
            validation_data=(X_val, [y_val_0, y_val_1, y_val_2, y_val_3, y_val_4, y_val_5]),
            callbacks=callbacks_list)

#Saving the model
model.save(os.path.join(model_dir, 'final_{}_{}'.format(NAME,t)))

Train on 4215 samples, validate on 543 samples
Epoch 1/500
4215/4215 [==============================] - 43s 10ms/step - loss: 0.1967 - dense_out_1_loss: 0.0377 - dense_out_2_loss: 0.0368 - dense_out_3_loss: 0.0317 - dense_out_4_loss: 0.0308 - dense_out_5_loss: 0.0307 - dense_out_6_loss: 0.0290 - dense_out_1_acc: 0.9893 - dense_out_2_acc: 0.9888 - dense_out_3_acc: 0.9905 - dense_out_4_acc: 0.9884 - dense_out_5_acc: 0.9912 - dense_out_6_acc: 0.9896 - val_loss: 4.3385 - val_dense_out_1_loss: 0.9970 - val_dense_out_2_loss: 0.9478 - val_dense_out_3_loss: 0.1076 - val_dense_out_4_loss: 0.1042 - val_dense_out_5_loss: 0.8831 - val_dense_out_6_loss: 1.2988 - val_dense_out_1_acc: 0.7790 - val_dense_out_2_acc: 0.7937 - val_dense_out_3_acc: 0.9834 - val_dense_out_4_acc: 0.9890 - val_dense_out_5_acc: 0.8361 - val_dense_out_6_acc: 0.7606
Avg_val_acc imporved from 0 to 0.8569674632935672
Saved model Models\best_ConvLSTM_model_1_1553165087
Epoch 2/500
4215/4215 [==============================] - 34s 8

Avg_val_acc imporved from 0.8686310604644598 to 0.8747697982267049
Saved model Models\best_ConvLSTM_model_1_1553165087
Epoch 11/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.2034 - dense_out_1_loss: 0.0449 - dense_out_2_loss: 0.0359 - dense_out_3_loss: 0.0356 - dense_out_4_loss: 0.0214 - dense_out_5_loss: 0.0255 - dense_out_6_loss: 0.0400 - dense_out_1_acc: 0.9848 - dense_out_2_acc: 0.9872 - dense_out_3_acc: 0.9872 - dense_out_4_acc: 0.9943 - dense_out_5_acc: 0.9917 - dense_out_6_acc: 0.9874 - val_loss: 3.8076 - val_dense_out_1_loss: 0.9411 - val_dense_out_2_loss: 0.8290 - val_dense_out_3_loss: 0.1386 - val_dense_out_4_loss: 0.0898 - val_dense_out_5_loss: 0.7070 - val_dense_out_6_loss: 1.1021 - val_dense_out_1_acc: 0.8029 - val_dense_out_2_acc: 0.8048 - val_dense_out_3_acc: 0.9687 - val_dense_out_4_acc: 0.9871 - val_dense_out_5_acc: 0.8729 - val_dense_out_6_acc: 0.7901
Epoch 12/500
4215/4215 [==============================] - 35s 8ms/step - loss: 0.1739 - dens

Avg_val_acc imporved from 0.8756906067651792 to 0.8787599775123772
Saved model Models\best_ConvLSTM_model_1_1553165087
Epoch 21/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.1865 - dense_out_1_loss: 0.0322 - dense_out_2_loss: 0.0362 - dense_out_3_loss: 0.0267 - dense_out_4_loss: 0.0247 - dense_out_5_loss: 0.0289 - dense_out_6_loss: 0.0378 - dense_out_1_acc: 0.9903 - dense_out_2_acc: 0.9886 - dense_out_3_acc: 0.9926 - dense_out_4_acc: 0.9900 - dense_out_5_acc: 0.9915 - dense_out_6_acc: 0.9891 - val_loss: 3.5504 - val_dense_out_1_loss: 0.7605 - val_dense_out_2_loss: 0.7146 - val_dense_out_3_loss: 0.1508 - val_dense_out_4_loss: 0.0943 - val_dense_out_5_loss: 0.7278 - val_dense_out_6_loss: 1.1024 - val_dense_out_1_acc: 0.8250 - val_dense_out_2_acc: 0.8379 - val_dense_out_3_acc: 0.9632 - val_dense_out_4_acc: 0.9908 - val_dense_out_5_acc: 0.8619 - val_dense_out_6_acc: 0.7937
Epoch 22/500
4215/4215 [==============================] - 35s 8ms/step - loss: 0.1840 - dens

4215/4215 [==============================] - 34s 8ms/step - loss: 0.1814 - dense_out_1_loss: 0.0339 - dense_out_2_loss: 0.0305 - dense_out_3_loss: 0.0299 - dense_out_4_loss: 0.0284 - dense_out_5_loss: 0.0296 - dense_out_6_loss: 0.0291 - dense_out_1_acc: 0.9900 - dense_out_2_acc: 0.9905 - dense_out_3_acc: 0.9907 - dense_out_4_acc: 0.9919 - dense_out_5_acc: 0.9905 - dense_out_6_acc: 0.9917 - val_loss: 3.7262 - val_dense_out_1_loss: 0.8246 - val_dense_out_2_loss: 0.8006 - val_dense_out_3_loss: 0.1582 - val_dense_out_4_loss: 0.0992 - val_dense_out_5_loss: 0.7129 - val_dense_out_6_loss: 1.1307 - val_dense_out_1_acc: 0.8287 - val_dense_out_2_acc: 0.8361 - val_dense_out_3_acc: 0.9705 - val_dense_out_4_acc: 0.9926 - val_dense_out_5_acc: 0.8508 - val_dense_out_6_acc: 0.7882
Epoch 32/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.1795 - dense_out_1_loss: 0.0430 - dense_out_2_loss: 0.0291 - dense_out_3_loss: 0.0276 - dense_out_4_loss: 0.0196 - dense_out_5_loss: 0.0327 - d

4215/4215 [==============================] - 34s 8ms/step - loss: 0.1663 - dense_out_1_loss: 0.0308 - dense_out_2_loss: 0.0333 - dense_out_3_loss: 0.0206 - dense_out_4_loss: 0.0172 - dense_out_5_loss: 0.0344 - dense_out_6_loss: 0.0300 - dense_out_1_acc: 0.9900 - dense_out_2_acc: 0.9917 - dense_out_3_acc: 0.9934 - dense_out_4_acc: 0.9936 - dense_out_5_acc: 0.9893 - dense_out_6_acc: 0.9903 - val_loss: 3.7729 - val_dense_out_1_loss: 0.8065 - val_dense_out_2_loss: 0.7544 - val_dense_out_3_loss: 0.1587 - val_dense_out_4_loss: 0.1046 - val_dense_out_5_loss: 0.7207 - val_dense_out_6_loss: 1.2279 - val_dense_out_1_acc: 0.8306 - val_dense_out_2_acc: 0.8453 - val_dense_out_3_acc: 0.9705 - val_dense_out_4_acc: 0.9908 - val_dense_out_5_acc: 0.8564 - val_dense_out_6_acc: 0.7864
Epoch 42/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.1645 - dense_out_1_loss: 0.0271 - dense_out_2_loss: 0.0293 - dense_out_3_loss: 0.0330 - dense_out_4_loss: 0.0205 - dense_out_5_loss: 0.0275 - d

...

4215/4215 [==============================] - 34s 8ms/step - loss: 0.0932 - dense_out_1_loss: 0.0185 - dense_out_2_loss: 0.0129 - dense_out_3_loss: 0.0132 - dense_out_4_loss: 0.0121 - dense_out_5_loss: 0.0166 - dense_out_6_loss: 0.0199 - dense_out_1_acc: 0.9934 - dense_out_2_acc: 0.9957 - dense_out_3_acc: 0.9960 - dense_out_4_acc: 0.9960 - dense_out_5_acc: 0.9948 - dense_out_6_acc: 0.9936 - val_loss: 2.6232 - val_dense_out_1_loss: 0.7100 - val_dense_out_2_loss: 0.4748 - val_dense_out_3_loss: 0.1240 - val_dense_out_4_loss: 0.0985 - val_dense_out_5_loss: 0.4750 - val_dense_out_6_loss: 0.7409 - val_dense_out_1_acc: 0.8858 - val_dense_out_2_acc: 0.8987 - val_dense_out_3_acc: 0.9797 - val_dense_out_4_acc: 0.9908 - val_dense_out_5_acc: 0.9263 - val_dense_out_6_acc: 0.8582
Epoch 452/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.0777 - dense_out_1_loss: 0.0153 - dense_out_2_loss: 0.0085 - dense_out_3_loss: 0.0149 - dense_out_4_loss: 0.0126 - dense_out_5_loss: 0.0127 - 

4215/4215 [==============================] - 34s 8ms/step - loss: 0.0859 - dense_out_1_loss: 0.0142 - dense_out_2_loss: 0.0198 - dense_out_3_loss: 0.0116 - dense_out_4_loss: 0.0117 - dense_out_5_loss: 0.0108 - dense_out_6_loss: 0.0179 - dense_out_1_acc: 0.9962 - dense_out_2_acc: 0.9948 - dense_out_3_acc: 0.9950 - dense_out_4_acc: 0.9969 - dense_out_5_acc: 0.9964 - dense_out_6_acc: 0.9957 - val_loss: 2.6648 - val_dense_out_1_loss: 0.6185 - val_dense_out_2_loss: 0.4680 - val_dense_out_3_loss: 0.0942 - val_dense_out_4_loss: 0.0953 - val_dense_out_5_loss: 0.4934 - val_dense_out_6_loss: 0.8954 - val_dense_out_1_acc: 0.9042 - val_dense_out_2_acc: 0.8969 - val_dense_out_3_acc: 0.9834 - val_dense_out_4_acc: 0.9926 - val_dense_out_5_acc: 0.9208 - val_dense_out_6_acc: 0.8527
Epoch 462/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.0996 - dense_out_1_loss: 0.0153 - dense_out_2_loss: 0.0188 - dense_out_3_loss: 0.0183 - dense_out_4_loss: 0.0143 - dense_out_5_loss: 0.0114 - 

4215/4215 [==============================] - 34s 8ms/step - loss: 0.0796 - dense_out_1_loss: 0.0139 - dense_out_2_loss: 0.0109 - dense_out_3_loss: 0.0156 - dense_out_4_loss: 0.0079 - dense_out_5_loss: 0.0149 - dense_out_6_loss: 0.0165 - dense_out_1_acc: 0.9953 - dense_out_2_acc: 0.9960 - dense_out_3_acc: 0.9948 - dense_out_4_acc: 0.9976 - dense_out_5_acc: 0.9957 - dense_out_6_acc: 0.9945 - val_loss: 2.7221 - val_dense_out_1_loss: 0.6134 - val_dense_out_2_loss: 0.4827 - val_dense_out_3_loss: 0.0849 - val_dense_out_4_loss: 0.0938 - val_dense_out_5_loss: 0.5512 - val_dense_out_6_loss: 0.8961 - val_dense_out_1_acc: 0.8858 - val_dense_out_2_acc: 0.9024 - val_dense_out_3_acc: 0.9834 - val_dense_out_4_acc: 0.9908 - val_dense_out_5_acc: 0.9171 - val_dense_out_6_acc: 0.8435
Epoch 472/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.0817 - dense_out_1_loss: 0.0202 - dense_out_2_loss: 0.0142 - dense_out_3_loss: 0.0129 - dense_out_4_loss: 0.0168 - dense_out_5_loss: 0.0073 - 

4215/4215 [==============================] - 34s 8ms/step - loss: 0.0889 - dense_out_1_loss: 0.0170 - dense_out_2_loss: 0.0166 - dense_out_3_loss: 0.0164 - dense_out_4_loss: 0.0132 - dense_out_5_loss: 0.0116 - dense_out_6_loss: 0.0141 - dense_out_1_acc: 0.9948 - dense_out_2_acc: 0.9953 - dense_out_3_acc: 0.9945 - dense_out_4_acc: 0.9960 - dense_out_5_acc: 0.9957 - dense_out_6_acc: 0.9960 - val_loss: 2.7791 - val_dense_out_1_loss: 0.6687 - val_dense_out_2_loss: 0.4924 - val_dense_out_3_loss: 0.1345 - val_dense_out_4_loss: 0.0742 - val_dense_out_5_loss: 0.4918 - val_dense_out_6_loss: 0.9176 - val_dense_out_1_acc: 0.8803 - val_dense_out_2_acc: 0.8932 - val_dense_out_3_acc: 0.9761 - val_dense_out_4_acc: 0.9908 - val_dense_out_5_acc: 0.9134 - val_dense_out_6_acc: 0.8490
Epoch 482/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.0886 - dense_out_1_loss: 0.0134 - dense_out_2_loss: 0.0233 - dense_out_3_loss: 0.0126 - dense_out_4_loss: 0.0112 - dense_out_5_loss: 0.0131 - 

4215/4215 [==============================] - 34s 8ms/step - loss: 0.0660 - dense_out_1_loss: 0.0137 - dense_out_2_loss: 0.0095 - dense_out_3_loss: 0.0099 - dense_out_4_loss: 0.0141 - dense_out_5_loss: 0.0075 - dense_out_6_loss: 0.0113 - dense_out_1_acc: 0.9950 - dense_out_2_acc: 0.9964 - dense_out_3_acc: 0.9972 - dense_out_4_acc: 0.9967 - dense_out_5_acc: 0.9976 - dense_out_6_acc: 0.9957 - val_loss: 2.8865 - val_dense_out_1_loss: 0.6629 - val_dense_out_2_loss: 0.5538 - val_dense_out_3_loss: 0.1108 - val_dense_out_4_loss: 0.0810 - val_dense_out_5_loss: 0.5403 - val_dense_out_6_loss: 0.9377 - val_dense_out_1_acc: 0.8913 - val_dense_out_2_acc: 0.9061 - val_dense_out_3_acc: 0.9761 - val_dense_out_4_acc: 0.9926 - val_dense_out_5_acc: 0.9079 - val_dense_out_6_acc: 0.8379
Epoch 492/500
4215/4215 [==============================] - 34s 8ms/step - loss: 0.0808 - dense_out_1_loss: 0.0144 - dense_out_2_loss: 0.0118 - dense_out_3_loss: 0.0144 - dense_out_4_loss: 0.0118 - dense_out_5_loss: 0.0145 - 

In [4]:
import tensorflow as tf

In [32]:
model = load_model('Models/best_ConvLSTM_model_1_1553165087')

In [33]:
(y_pred_0, y_pred_1, y_pred_2, y_pred_3, y_pred_4, y_pred_5) = model.predict(X_test)

In [34]:
y_pred_0 = np.argmax(y_pred_0, axis=1)
y_pred_1 = np.argmax(y_pred_1, axis=1)
y_pred_2 = np.argmax(y_pred_2, axis=1)
y_pred_3 = np.argmax(y_pred_3, axis=1)
y_pred_4 = np.argmax(y_pred_4, axis=1)
y_pred_5 = np.argmax(y_pred_5, axis=1)

In [35]:
acc = 0
true = []
pred = []
for i,code in enumerate(y_test_coded):
    pred.append(labels_to_text([y_pred_0[i],y_pred_1[i],y_pred_2[i],y_pred_3[i],y_pred_4[i],y_pred_5[i]]))
    true.append(labels_to_text(code))
    if(pred[i] == true[i]):
        acc = acc+1
print(acc)

734


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test_coded[:,0],y_pred_0)
cm = cm/cm.sum(axis=0)
df_cm = pd.DataFrame(cm, index = [i for i in ['Left','Right','Legs','Tongue']],
                  columns = [i for i in ['Left','Right','Legs','Tongue']])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [36]:
from sklearn.metrics import accuracy_score

accuracy_0 = accuracy_score(y_test_coded[:,0],y_pred_0)
accuracy_1 = accuracy_score(y_test_coded[:,1],y_pred_1)
accuracy_2 = accuracy_score(y_test_coded[:,2],y_pred_2)
accuracy_3 = accuracy_score(y_test_coded[:,3],y_pred_3)
accuracy_4 = accuracy_score(y_test_coded[:,4],y_pred_4)
accuracy_5 = accuracy_score(y_test_coded[:,5],y_pred_5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)
print(accuracy_5)

0.8654970760233918
0.9022556390977443
0.9598997493734336
0.9949874686716792
0.9097744360902256
0.8663324979114453


In [37]:
ed = []
for t,p in zip(true,pred):
    ed.append(editdistance.eval(t,p))

Counter(ed)

Counter({0: 734, 1: 356, 2: 87, 3: 13, 4: 5, 5: 1, 6: 1})

In [ ]:
Counter({1: 443, 2: 289, 0: 322, 4: 26, 3: 109, 5: 7, 6: 1})

In [ ]:
Counter({0: 736, 1: 362, 2: 78, 3: 16, 4: 3, 5: 1, 6: 1})
